In [12]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import xgboost as xgb

In [13]:
df = pd.read_csv('LoanStats3a 2.csv',low_memory=False,skiprows=1)

In [14]:
df.isnull().any()  # 检查 null

id                                            True
member_id                                     True
loan_amnt                                     True
funded_amnt                                   True
funded_amnt_inv                               True
term                                          True
int_rate                                      True
installment                                   True
grade                                         True
sub_grade                                     True
emp_title                                     True
emp_length                                    True
home_ownership                                True
annual_inc                                    True
verification_status                           True
issue_d                                       True
loan_status                                   True
pymnt_plan                                    True
url                                           True
desc                           

In [15]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.shape

(42538, 144)

In [17]:
df = df.iloc[:,2:111]  # 删掉很多空的列
empty_cols = [i for i in range(45,72)]   # 删除更多的列
df = df.drop(df.columns[empty_cols],axis=1)
df.shape

(42538, 82)

In [18]:
df = df[(df['loan_status']=="Fully Paid") | (df['loan_status']=="Charged Off")]
df['loan_status'] = df['loan_status'].map({'Fully Paid':0, 'Charged Off':1})


In [19]:
df=df.dropna(axis=1) #340000 is minimum number of non-NA values
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,delinq_amnt
0,5000.0,5000.0,4975.000000,36 months,10.65%,162.87,B,B2,RENT,24000.00,...,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.0000,171.62,0.0
1,2500.0,2500.0,2500.000000,60 months,15.27%,59.83,C,C4,RENT,30000.00,...,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.90,1.1100,119.66,0.0
2,2400.0,2400.0,2400.000000,36 months,15.96%,84.33,C,C5,RENT,12252.00,...,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.0000,649.91,0.0
3,10000.0,10000.0,10000.000000,36 months,13.49%,339.31,C,C1,RENT,49200.00,...,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.0000,357.48,0.0
4,3000.0,3000.0,3000.000000,60 months,12.69%,67.79,B,B5,RENT,80000.00,...,0.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.00,0.0000,67.30,0.0
5,5000.0,5000.0,5000.000000,36 months,7.90%,156.46,A,A4,RENT,36000.00,...,0.0,5632.210000,5632.21,5000.00,632.21,0.00,0.00,0.0000,161.03,0.0
6,7000.0,7000.0,7000.000000,60 months,15.96%,170.08,C,C5,RENT,47004.00,...,0.0,10137.840008,10137.84,7000.00,3137.84,0.00,0.00,0.0000,1313.76,0.0
7,3000.0,3000.0,3000.000000,36 months,18.64%,109.43,E,E1,RENT,48000.00,...,0.0,3939.135294,3939.14,3000.00,939.14,0.00,0.00,0.0000,111.34,0.0
8,5600.0,5600.0,5600.000000,60 months,21.28%,152.39,F,F2,OWN,40000.00,...,0.0,647.500000,647.50,162.02,294.94,0.00,190.54,2.0900,152.39,0.0
9,5375.0,5375.0,5350.000000,60 months,12.69%,121.45,B,B5,RENT,15000.00,...,0.0,1484.590000,1477.70,673.48,533.42,0.00,277.69,2.5200,121.45,0.0


In [20]:
df_grade = df['grade'].str.get_dummies().add_prefix('grade: ')
# 把类型独热编码
df_subgrad = df['sub_grade'].apply(str).str.get_dummies().add_prefix('sub_grade: ')
df_home = df['home_ownership'].apply(str).str.get_dummies().add_prefix('home_ownership: ')
df_addr = df['addr_state'].apply(str).str.get_dummies().add_prefix('addr_state: ')
df_term = df['term'].apply(str).str.get_dummies().add_prefix('term: ')

# 添加独热编码数据列
df = pd.concat([df, df_grade, df_subgrad, df_home, df_addr, df_term], axis=1)
# 去除独热编码对应的原始列
df = df.drop(['grade', 'sub_grade', 'home_ownership', 'addr_state', 'int_rate', 'term', 'zip_code','purpose','initial_list_status','initial_list_status','pymnt_plan','issue_d','earliest_cr_line','verification_status'], axis=1)


In [21]:
df.dtypes

loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
installment                float64
annual_inc                 float64
loan_status                  int64
dti                        float64
delinq_2yrs                float64
inq_last_6mths             float64
open_acc                   float64
pub_rec                    float64
revol_bal                  float64
total_acc                  float64
out_prncp                  float64
out_prncp_inv              float64
total_pymnt                float64
total_pymnt_inv            float64
total_rec_prncp            float64
total_rec_int              float64
total_rec_late_fee         float64
recoveries                 float64
collection_recovery_fee    float64
last_pymnt_amnt            float64
delinq_amnt                float64
grade: A                     int64
grade: B                     int64
grade: C                     int64
grade: D                     int64
grade: E            

In [22]:
df.corr()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,addr_state: TX,addr_state: UT,addr_state: VA,addr_state: VT,addr_state: WA,addr_state: WI,addr_state: WV,addr_state: WY,term: 36 months,term: 60 months
loan_amnt,1.000000,0.981544,0.940160,0.930209,0.271340,0.050035,0.067136,-0.031484,0.009442,0.177216,...,0.018017,0.000509,0.008110,-0.009365,-0.005137,0.000804,-0.003334,-0.001011,-0.362123,0.362123
funded_amnt,0.981544,1.000000,0.958568,0.956108,0.267130,0.047299,0.067035,-0.031948,0.009383,0.175515,...,0.017005,0.000302,0.009806,-0.009734,-0.003134,0.001366,-0.001937,-0.000496,-0.341611,0.341611
funded_amnt_inv,0.940160,0.958568,1.000000,0.905101,0.254536,0.028681,0.075434,-0.038069,-0.005550,0.163022,...,0.014190,-0.001340,0.009917,-0.010355,-0.005162,0.001156,0.001664,0.001446,-0.361885,0.361885
installment,0.930209,0.956108,0.905101,1.000000,0.271119,0.023566,0.054730,-0.019309,0.009894,0.172885,...,0.020659,-0.000718,0.008720,-0.008944,0.000149,-0.002616,-0.005236,0.001775,-0.102976,0.102976
annual_inc,0.271340,0.267130,0.254536,0.271119,1.000000,-0.041168,-0.122537,0.022985,0.034226,0.158178,...,0.019393,-0.004621,0.014772,-0.008669,-0.006953,-0.010121,-0.014719,-0.008909,-0.046809,0.046809
loan_status,0.050035,0.047299,0.028681,0.023566,-0.041168,1.000000,0.041984,0.020197,0.071325,-0.010199,...,-0.019222,0.003656,-0.008817,-0.003312,0.003499,-0.001719,-0.004564,-0.012336,-0.149151,0.149151
dti,0.067136,0.067035,0.075434,0.054730,-0.122537,0.041984,1.000000,-0.034562,0.001452,0.288087,...,0.025194,-0.000321,0.002183,0.001113,-0.004200,0.001867,0.017600,0.006149,-0.082822,0.082822
delinq_2yrs,-0.031484,-0.031948,-0.038069,-0.019309,0.022985,0.020197,-0.034562,1.000000,0.007854,0.011822,...,-0.000934,0.001301,0.001295,0.005673,-0.012208,-0.001628,-0.003023,0.006539,-0.006405,0.006405
inq_last_6mths,0.009442,0.009383,-0.005550,0.009894,0.034226,0.071325,0.001452,0.007854,1.000000,0.091554,...,0.016715,-0.001777,-0.011775,0.010254,-0.011055,0.039822,0.007475,0.007140,-0.040847,0.040847
open_acc,0.177216,0.175515,0.163022,0.172885,0.158178,-0.010199,0.288087,0.011822,0.091554,1.000000,...,0.028681,0.001197,0.007359,-0.008670,-0.016494,0.003032,0.001713,-0.007688,-0.050464,0.050464


In [24]:
# 准备数据
X = df.drop('loan_status', axis=1)
y = df['loan_status']
print (X.shape, y.shape)

(39786, 122) (39786,)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [28]:
xg_classifier = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [29]:
xg_classifier.fit(X_train,y_train)
xg_classifier.score(X_test, y_test)

0.9889419452123649